In [ ]:
from transformers import AutoTokenizer, AutoModel
from transformers import pipeline
import numpy as np
import pandas as pd

from pythainlp import word_tokenize
import re
from pythainlp.corpus import thai_stopwords

from simpletransformers.classification import ClassificationModel, ClassificationArgs
import sklearn

import torch

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
thai_stop_words = thai_stopwords()

In [ ]:
pretrained = "monsoon-nlp/bert-base-thai"
name = pretrained

tokenizer = AutoTokenizer.from_pretrained(name, use_fast=True)

In [ ]:
model = AutoModel.from_pretrained(name)

# Preprocessing

In [ ]:
text = ["สวัสดีครับ ฉันชื่อโก้ ฉันชอบกินกล้วย ฉันเป็นคนเก่ง"]

In [ ]:
sent_id = tokenizer.batch_encode_plus(text, padding=True)

In [ ]:
print(sent_id)

# Fill Mask

In [ ]:
unmasker = pipeline('fill-mask', model=name, tokenizer=tokenizer)

In [ ]:
unmasker("โก้ ชอบ [MASK] เงิน")

 # Text Classification

In [ ]:
df = pd.read_csv('main_suicidal_data.csv')
df.head(5)

In [ ]:
df['Label_Enc'] = df['Label (Specialist)'].str.replace('Level 1','Low Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 2','Mid Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 3','Mid Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 4','High Level Depress')
df['Label_Enc'] = df['Label_Enc'].str.replace('Level 5','High Level Depress')

In [ ]:
conditions = [
    (df['Label_Enc'] == 'Other'),
    (df['Label_Enc'] == 'Low Level Depress'),
    (df['Label_Enc'] == 'Mid Level Depress'),
    (df['Label_Enc'] == 'High Level Depress'),
    ]

values = ['0', '1', '2', '3']

df['Label_Enc'] = np.select(conditions, values)


In [ ]:
def replace_url(text):
    URL_PATTERN = r"""(?i)\b((?:https?:(?:/{1,3}|[a-z0-9%])|[a-z0-9.\-]+[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)/)(?:[^\s()<>{}
]+|[^\s()]*?\)|)+(?:[^\s()]*?\)||[^\s`!()
{};:'".,<>?«»“”‘’])|(?:(?<!@)[a-z0-9]+(?:[.\-][a-z0-9]+)*[.](?:com|net|org|edu|gov|mil|aero|asia|biz|cat|coop|info|int|jobs|mobi|museum|name|post|pro|tel|travel|xxx|ac|ad|ae|af|ag|ai|al|am|an|ao|aq|ar|as|at|au|aw|ax|az|ba|bb|bd|be|bf|bg|bh|bi|bj|bm|bn|bo|br|bs|bt|bv|bw|by|bz|ca|cc|cd|cf|cg|ch|ci|ck|cl|cm|cn|co|cr|cs|cu|cv|cx|cy|cz|dd|de|dj|dk|dm|do|dz|ec|ee|eg|eh|er|es|et|eu|fi|fj|fk|fm|fo|fr|ga|gb|gd|ge|gf|gg|gh|gi|gl|gm|gn|gp|gq|gr|gs|gt|gu|gw|gy|hk|hm|hn|hr|ht|hu|id|ie|il|im|in|io|iq|ir|is|it|je|jm|jo|jp|ke|kg|kh|ki|km|kn|kp|kr|kw|ky|kz|la|lb|lc|li|lk|lr|ls|lt|lu|lv|ly|ma|mc|md|me|mg|mh|mk|ml|mm|mn|mo|mp|mq|mr|ms|mt|mu|mv|mw|mx|my|mz|na|nc|ne|nf|ng|ni|nl|no|np|nr|nu|nz|om|pa|pe|pf|pg|ph|pk|pl|pm|pn|pr|ps|pt|pw|py|qa|re|ro|rs|ru|rw|sa|sb|sc|sd|se|sg|sh|si|sj|Ja|sk|sl|sm|sn|so|sr|ss|st|su|sv|sx|sy|sz|tc|td|tf|tg|th|tj|tk|tl|tm|tn|to|tp|tr|tt|tv|tw|tz|ua|ug|uk|us|uy|uz|va|vc|ve|vg|vi|vn|vu|wf|ws|ye|yt|yu|za|zm|zw)\b/?(?!@)))"""
    return re.sub(URL_PATTERN, 'xxurl', text)

def replace_rep(text):
    def _replace_rep(m):
        c,cc = m.groups()
        return f'{c}xxrep'
    re_rep = re.compile(r'(\S)(\1{2,})')
    return re_rep.sub(_replace_rep, text)

def process_text(text):
    res = text.lower().strip()
    res = replace_url(res)
    res = replace_rep(res)

    res = [word for word in word_tokenize(res) if word and not re.search(pattern=r"\s+", string=word)]
    return res

In [ ]:
df['Tweet'] = df['Tweet'].map(lambda x: ' '.join(process_text(x)))

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
train_data["Label_Enc"].hist()

In [ ]:
train_df, val_df = train_test_split(train_data, test_size=0.15, random_state=23)

In [ ]:
train_df["text"] = train_df["Tweet"]

val_df["text"] = val_df["Tweet"]

test_data["text"] = test_data["Tweet"]

In [ ]:
num_labels = len(np.unique(np.array(train_df["Label_Enc"])))

In [ ]:
print(num_labels)

In [ ]:
model_args = {
    "reprocess_input_data": True,
    'train_batch_size':32,
    "use_early_stopping": True,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "early_stopping_metric_minimize": False,
    "early_stopping_patience": 5,
    "evaluate_during_training_steps": 500,
    "fp16": False,
    "num_train_epochs":3,
    "overwrite_output_dir":True
}

model = ClassificationModel(
     "bert", 
     name,
     use_cuda=torch.cuda.is_available(), 
     args=model_args, 
     num_labels=num_labels,
)
model.train_model(train_df, show_running_loss = True, eval_df=val_df, acc=sklearn.metrics.accuracy_score)